In [1]:
import pandas as pd

In [2]:
df_hot = pd.read_excel('df_cat.xlsx')

In [3]:
df_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10775 entries, 0 to 10774
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      10775 non-null  int64 
 1   national_catid  10775 non-null  int64 
 2   categroy_name   10775 non-null  object
dtypes: int64(2), object(1)
memory usage: 252.7+ KB


In [18]:
df_hot = df_hot.sample(5000)

In [8]:
df_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10775 entries, 0 to 10774
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   national_catid  10775 non-null  int64 
 1   category_name   10775 non-null  object
dtypes: int64(1), object(1)
memory usage: 168.5+ KB


In [46]:
df_hot.to_excel('gpt_5k.xlsx' , index=False)

In [4]:
df_hot.rename(columns={'categroy_name' : 'category_name'} , inplace = True)

In [6]:
df_hot.to_excel('top_5k_category.xlsx' , index=False)

In [5]:
df_hot.columns

Index(['Unnamed: 0', 'national_catid', 'category_name'], dtype='object')

In [6]:
df_hot = df_hot[['national_catid', 'category_name']]

In [7]:
df_hot.head()

,national_catid,category_name
0,10050521,Body Massage Centres
1,11010608,Massage Centres For Men
2,10272436,Interior Designers
3,10076456,Car Rental
4,10042600,Beauty Spas


In [ ]:
# step 1 run this cell to import the library and access the gpt api
import json
import re
import faiss
import torch
import requests
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import re
# ----------------------- SETUP -----------------------



In [19]:
# step3 to call the gpt api
def get_response(prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    payload = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1600
    }
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        return response.json().get("choices", [{}])[0].get("message", {}).get("content", "")
        # return response.json()
    except Exception as e:
        return f"Error: {e}"


In [11]:
# step4 prompt 1
def prompt1(history_conversation , original_input):
    history_text = ""
    categories = ""
    
    # Build categories string
    for cat in df_hot['category_name']:  # <-- fixed typo here
        categories += f"{cat}, "
    
    # Build history text
    for i, (question, reply) in enumerate(history_conversation.items(), start=1):
        history_text += f"{i}. You asked: {question}\n   User replied: {reply}\n"

    # print(history_text)
    prompt2 = f"""
   You are an intelligent assistant helping Indian users find the right local service or business. Your job is to extract details from the user’s query like category , area and city:
  Input Context
    Previous clarifications: "{history_text}"
    Reference Justdial categories: "{categories}"
    Extraction Priority
    Category (Highest priority): Used Justdial categories as refrence to suggest most similar category which fulfill the user need also please ensure that suggested category should present on the justdial platform.
    City (Second priority): Extract the Indian city where service is required.
    Area (Third priority): Extract neighbourhood, locality, colony, or sub-part of the city.
    - If all three details (category, area, city) are confidently identified and specific and there is no missing value and all value is filled  then only return in this format:
      Return only this JSON format (no explanation text):
          {{"category": "category_name", "area": "area_name", "city": "city_name"}}
    If any detail is missing or unclear, output a direct question to get that detail.
    If any detail is missing or unclear, do not output JSON. Instead, ask a direct clarification question such as:
    "Could you please specify the city where you need this service?"
    "Which area in Delhi are you searching in?"
    "Are you looking for a train ticket agent, bus booking, or flight booking?"     
    """
    return prompt2

In [12]:
df_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10775 entries, 0 to 10774
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   national_catid  10775 non-null  int64 
 1   category_name   10775 non-null  object
dtypes: int64(1), object(1)
memory usage: 168.5+ KB


In [13]:
# step 9
def extract_json(response):
    match = re.search(r'\{.*?\}', response, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None
    return None


In [14]:
def preprocess_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # keep only letters, numbers, spaces
    return text.strip()

In [15]:
  # step 11
  not_field = {
    "N/A", "NA", "n/a", "na", "null", "NULL", "", "-", "none", "None",
    "not available", "not applicable", "unspecified", "unknown",
    "not_available", "not_applicable", "not_provided", "not_supplied",
    "not_captured", "not_recorded", "not_collected", "not_entered",
    "not_disclosed", "not_shared", "not_furnished", "not_known",
    "not_identified", "not_determined", "undetermined", "missing",
    "missing_value", "missing_data", "no_data", "no_response",
    "no_answer", "no_result", "empty_value", "blank_value",
    "null_value", "withheld", "redacted", "confidential",
    "pii_removed", "policy_restricted", "compliance_restricted",
    "restricted", "forbidden", "blocked", "safety_blocked",
    "content_blocked", "ambiguous", "unclear", "vague",
    "needs_clarification", "insufficient_context", "insufficient_detail",
    "low_confidence","not provided"
        }

In [35]:

history_conversation1 = {}

In [42]:
prompt2 = prompt1(history_conversation1, 'I am looking for totels')
response = get_response(prompt2)

In [43]:
response

{'choices': [{'content_filter_results': {'hate': {'filtered': False,
     'severity': 'safe'},
    'protected_material_code': {'filtered': False, 'detected': False},
    'protected_material_text': {'filtered': False, 'detected': False},
    'self_harm': {'filtered': False, 'severity': 'safe'},
    'sexual': {'filtered': False, 'severity': 'safe'},
    'violence': {'filtered': False, 'severity': 'safe'}},
   'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'annotations': [],
    'content': 'Could you please specify the city where you need this service?',
    'refusal': None,
    'role': 'assistant'}}],
 'created': 1756792683,
 'id': 'chatcmpl-CBEJnz9TYDAg61HQOrnYvKYYWzeLt',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'prompt_filter_results': [{'prompt_index': 0,
   'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
    'jailbreak': {'filtered': False, 'detected': False},
    'self_harm': {'filtered': False, 'seve

In [16]:
# step 12
# main function handle all conversation and follow up question
def main_function(response, area_1="", city_1="" , start=False):
    first_user = input(f"Assitant : {response}\nUser : ")
    history_conversation = {}
    a = response
    a = a.lower()
    if start == True:
      first_user +=f"area is {area_1} , and city is {city_1} Please used this only as area and city not ask the question related area and city but ask question on categories for clarity"
    history_conversation[a] = first_user.lower()
    df_cat_res = pd.DataFrame()
    df_area_res = pd.DataFrame()
    cat = ''
    area = ''
    city = ''
    cnt=0
    # Adjusted threshold condition for better clarity
    while (cnt < 7):
        # Assuming prompt_fun1 generates the query for extracting data
        prompt2 = prompt1(history_conversation, first_user)
        response = get_response(prompt2)
        # print(response)
        parsed = extract_json(response)
        if parsed:
            cat = parsed.get("category")
            area = parsed.get("area")
            city = parsed.get("city")

            # If any of the parsed data is missing, skip and ask for more clarification
            cat_clean = preprocess_text(cat)
            area_clean = preprocess_text(area)
            city_clean = preprocess_text(city)

            # Preprocess not_field set
            not_field_clean = {preprocess_text(x) for x in not_field}

            # Now check
            if (not cat_clean or cat_clean in not_field_clean or
                not area_clean or area_clean in not_field_clean or
                not city_clean or city_clean in not_field_clean):

                missing_data = []
                if not cat_clean or cat_clean in not_field_clean:
                    missing_data.append("Category")
                if not area_clean or area_clean in not_field_clean:
                    missing_data.append("Area")
                if not city_clean or city_clean in not_field_clean:
                    missing_data.append("City")
                missing_str = ', '.join(missing_data)
                response = f"We are unable to get your {missing_str}, Please specify it with city."
                user_input = input(f"Assitant : {response}\nUser : ")
                history_conversation[response] = user_input
                cnt += 1
                continue   # Re-run the loop for further input            
            print(f"Gpt Extracted:\nCategory: {cat}\nArea: {area}\nCity: {city}")
            return cat , area , city
        else :
            user = input(f"Assitant : {response}\nUser : ")
            history_conversation[response] = user
            cnt+=1
    print(f"Gpt Extracted:\nCategory: {cat}\nArea: {area}\nCity: {city}")
    return cat , area , city
           

In [17]:
# step 14
# This final function which handle the starting and ending of conversation
def whatshapp_chat():
  end = False
  response = 'How can I help you ?'
  area = ''
  city = ''
  start=False
  while end!=True:
    try:
      cat_name, main_area , city = main_function(response , area , city , start)
      start = True
      temp = input(f"Do you want to end the conversation ? (yes/no): \n").lower()
      if temp == "yes":
        end = True
        break
    except Exception as e:
      print(f"An error occurred: {e}")
      end = True

In [18]:
whatshapp_chat()

Assitant : How can I help you ?
User :   my sister getting married khana bhanana ke liya chahiye


An error occurred: expected string or bytes-like object, got 'dict'


In [20]:
whatshapp_chat()

Assitant : How can I help you ?
User :  my sister getting married khana bhanana ke liya chahiye
Assitant : Could you please specify the city where you need this service?
User :  bangalore
Assitant : Could you please specify the area in Bangalore where you need this catering service?
User :  peenya


KeyboardInterrupt: Interrupted by user